## Llama 7B

Using the LangServe API

In [1]:
import re
from tqdm import tqdm
tqdm.pandas()

In [2]:
from langchain.chains import LLMChain

# llm
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp

# Prompt
from langchain.chains.prompt_selector import ConditionalPromptSelector
from langchain.prompts import PromptTemplate

# Parser
from langchain_core.output_parsers import StrOutputParser

In [3]:
prompt = PromptTemplate(
    input_variables=["headline"],
    template="""<<SYS>> \n You are an assistant tasked in geo-locating \
this news article. \n <</SYS>> \n\n [INST] Generate a SHORT response \
of where you think this article is talking about. BE SPECIFIC AND CONCISE AS POSSIBLE. Give your response in the following format: \
1.Y/N indicating whether the article is talking about a region of Boston \n 2.The specific location within the city you got if you got Y in the first question. \
If you do not know, please say 'I do not know' or 'Not enough information'. \n\n
Headline: \n\n {headline} \n\n [/INST]""",
)

In [4]:
llama_model_path = "./models/llama_7B/llama-2-7b-chat.Q4_K_M.gguf"

In [5]:
llm = LlamaCpp(
    model_path=llama_model_path,
    n_gpu_layers=1,
    n_batch=1024,
    n_ctx=2048,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/llama_7B/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u

In [6]:
output_parser = StrOutputParser()

In [7]:
chain = prompt | llm | output_parser

## Benchmarking

Here we benchmark how well the llama_7B does against some real data and our predictions

In [8]:
import pandas as pd
from bs4 import BeautifulSoup

In [9]:
#sample_data_dir = "./sample_data/articles_data.csv" # Our Current Database Articles
sample_data_dir = "./eval_dataset/Singer_Evaluation_Set.csv" # Singer Evaluation Set

In [10]:
raw_df = pd.read_csv(sample_data_dir)

In [11]:
raw_df.columns

Index(['Singer notes ', '_id', 'neighborhoods[0]', 'neighborhoods[1]',
       'neighborhoods[2]', 'neighborhoods[3]', 'neighborhoods[4]',
       'neighborhoods[5]', 'position_section', 'tracts[0]', 'tracts[1]',
       'tracts[2]', 'tracts[3]', 'tracts[4]', 'tracts[5]', 'tracts[6]',
       'author', 'body', 'content_id', 'hl1', 'hl2', 'pub_date', 'pub_name',
       'link', 'openai_labels[0]', 'Singer labels', 'userID', 'uploadID',
       'dateSum'],
      dtype='object')

In [12]:
df = pd.concat([raw_df['hl1'], raw_df['body'], raw_df['neighborhoods[0]']], axis=1)

In [13]:
df

,hl1,body,neighborhoods[0]
0,Tonight‚Äôs Celebration of ‚ÄúEveryone‚Äôs Fav...,Tonight‚Äôs celebrity and family-filled progra...,Allston
1,"Winter storm snarls travel, knocks out power f...",A storm making its way up the East Coast broug...,East Boston
2,A Walk Down the Block: Nubian Square's economi...,"<bsp-accordion data-state=""{&quot;cms.site.own...",Roxbury
3,Mass. makes moves to figure out what happens a...,As the state nears its threshold level for hou...,South Boston Waterfront
4,‚ÄòWe need to be ready‚Äô: CDC gives Northeast...,Northeastern University will act as a national...,Fenway
5,"Faced with a lack of funding, collegiate cheer...","For 10 Sundays each year, members of the Bridg...",Fenway
6,How to know who deserves your vote in a news d...,Deception in politics is a tale as old as time...,Downtown
7,Wu ‚Äòconfident‚Äô that new Mass. and Cass pla...,Boston Mayor Michelle Wu said late last week t...,Roxbury
8,Healey anticipates ‚Äòhard calls‚Äô as emergen...,Gov. Maura Healey says she‚Äôs bracing to make...,Downtown
9,"With need surging, behavioral urgent care cent...",East Boston Neighborhood Health Center held an...,East Boston


In [14]:
df["llama_pred"] = "N/A"

In [15]:
df

,hl1,body,neighborhoods[0],llama_pred
0,Tonight‚Äôs Celebration of ‚ÄúEveryone‚Äôs Fav...,Tonight‚Äôs celebrity and family-filled progra...,Allston,N/A
1,"Winter storm snarls travel, knocks out power f...",A storm making its way up the East Coast broug...,East Boston,N/A
2,A Walk Down the Block: Nubian Square's economi...,"<bsp-accordion data-state=""{&quot;cms.site.own...",Roxbury,N/A
3,Mass. makes moves to figure out what happens a...,As the state nears its threshold level for hou...,South Boston Waterfront,N/A
4,‚ÄòWe need to be ready‚Äô: CDC gives Northeast...,Northeastern University will act as a national...,Fenway,N/A
5,"Faced with a lack of funding, collegiate cheer...","For 10 Sundays each year, members of the Bridg...",Fenway,N/A
6,How to know who deserves your vote in a news d...,Deception in politics is a tale as old as time...,Downtown,N/A
7,Wu ‚Äòconfident‚Äô that new Mass. and Cass pla...,Boston Mayor Michelle Wu said late last week t...,Roxbury,N/A
8,Healey anticipates ‚Äòhard calls‚Äô as emergen...,Gov. Maura Healey says she‚Äôs bracing to make...,Downtown,N/A
9,"With need surging, behavioral urgent care cent...",East Boston Neighborhood Health Center held an...,East Boston,N/A


In [16]:
func_clean_html = lambda x: BeautifulSoup(x, "html.parser").get_text()
df['body'] = df['body'].progress_apply(func_clean_html)
df['hl1'] = df['hl1'].progress_apply(func_clean_html)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 23196.68it/s]


In [17]:
# Get the token length, llama is maxed out at 2048 
df['token_length'] = df['body'].progress_apply(lambda x: len(x))

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 45153.99it/s]


In [18]:
# DF Filtering
# length_criteria_deletion = df['token_length'] >= 2045
# df = df[~length_criteria_deletion] # Invert it so that True becomes false so it marks for deletion

In [19]:
df

,hl1,body,neighborhoods[0],llama_pred,token_length
0,Tonight‚Äôs Celebration of ‚ÄúEveryone‚Äôs Fav...,Tonight‚Äôs celebrity and family-filled progra...,Allston,N/A,3411
1,"Winter storm snarls travel, knocks out power f...",A storm making its way up the East Coast broug...,East Boston,N/A,2478
2,A Walk Down the Block: Nubian Square's economi...,"Yesterday, we took a walk down the block in Ro...",Roxbury,N/A,2928
3,Mass. makes moves to figure out what happens a...,As the state nears its threshold level for hou...,South Boston Waterfront,N/A,4245
4,‚ÄòWe need to be ready‚Äô: CDC gives Northeast...,Northeastern University will act as a national...,Fenway,N/A,1528
5,"Faced with a lack of funding, collegiate cheer...","For 10 Sundays each year, members of the Bridg...",Fenway,N/A,7539
6,How to know who deserves your vote in a news d...,Deception in politics is a tale as old as time...,Downtown,N/A,8300
7,Wu ‚Äòconfident‚Äô that new Mass. and Cass pla...,Boston Mayor Michelle Wu said late last week t...,Roxbury,N/A,4422
8,Healey anticipates ‚Äòhard calls‚Äô as emergen...,Gov. Maura Healey says she‚Äôs bracing to make...,Downtown,N/A,4609
9,"With need surging, behavioral urgent care cent...",East Boston Neighborhood Health Center held an...,East Boston,N/A,3296


In [20]:
predict_llama = lambda col: chain.invoke({"headline": col['hl1'], "body": col['body']})
df['llama_pred'] = df.progress_apply(predict_llama, axis=1)

  0%|                                                                                                          | 0/27 [00:00<?, ?it/s]

  Sure, I'd be happy to help! Based on the information provided in the article, I would estimate that the event being described is taking place in the North End neighborhood of Boston, Massachusetts. Here's my reasoning:
1. Y - The article does not provide any explicit clues suggesting that the event is taking place outside of Boston or in a different city. The mention of "tonight" and "Boston" suggest that the event is happening in the present and in a specific location within the city.
2. The specific location within the city I got is: North End. Based on the information provided in the article, it seems likely that the event is taking place in the North End neighborhood of Boston, which is known for its Italian American culture and cuisine.


llama_print_timings:        load time =    8183.60 ms
llama_print_timings:      sample time =      13.97 ms /   163 runs   (    0.09 ms per token, 11665.35 tokens per second)
llama_print_timings: prompt eval time =    8183.22 ms /   181 tokens (   45.21 ms per token,    22.12 tokens per second)
llama_print_timings:        eval time =    9454.67 ms /   162 runs   (   58.36 ms per token,    17.13 tokens per second)
llama_print_timings:       total time =   17972.80 ms /   343 tokens
  7%|███████▎                                                                                          | 2/27 [00:17<03:44,  8.99s/it]Llama.generate: prefix-match hit


  Based on the information provided in the article, here is my response:
1. Y - The article is talking about a region within Massachusetts, specifically the area affected by the winter storm.
2. Cambridge, Massachusetts - Based on the language used in the article, it appears that the storm affected the Cambridge area specifically, as mentioned in the phrase "hundreds of thousands across Massachusetts, including Cambridge."


llama_print_timings:        load time =    8183.60 ms
llama_print_timings:      sample time =       7.32 ms /    84 runs   (    0.09 ms per token, 11469.14 tokens per second)
llama_print_timings: prompt eval time =    1007.24 ms /    24 tokens (   41.97 ms per token,    23.83 tokens per second)
llama_print_timings:        eval time =    5019.91 ms /    83 runs   (   60.48 ms per token,    16.53 tokens per second)
llama_print_timings:       total time =    6196.39 ms /   107 tokens
 11%|██████████▉                                                                                       | 3/27 [00:24<03:07,  7.83s/it]Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    8183.60 ms
llama_print_timings:      sample time =       7.18 ms /    83 runs   (    0.09 ms per token, 11561.50 tokens per second)
llama_print_timings: prompt eval time =     749.25 ms /    17 tokens (   44.07 ms per token,    22.69 tokens per second)
llama_print_timings:        eval time =    4629.42 ms /    82 runs   (   56.46 ms per token,    17.71 tokens per second)
llama_print_timings:       total time =    5544.73 ms /    99 tokens
 96%|█████████████████████████████████████████████████████████████████████████████████████████████▍   | 26/27 [02:29<00:04,  4.76s/it]Llama.generate: prefix-match hit


  Based on the information provided in the article, I believe the location being referred to is Boston, Massachusetts.

1. Y - The article explicitly states that the funding for the LGBTQ+ community projects was stripped by the Appropriations Committee, which suggests that the location being referred to is within the city of Boston.
2. The specific location within Boston that the article mentions is the "Boston" project, which implies that the article is referring to a specific area or neighborhood within the city, such as Downtown Boston, the South End, or the Fenway neighborhood. I do not know the exact location within Boston without further context or information.


llama_print_timings:        load time =    8183.60 ms
llama_print_timings:      sample time =      11.94 ms /   140 runs   (    0.09 ms per token, 11729.22 tokens per second)
llama_print_timings: prompt eval time =    1757.72 ms /    32 tokens (   54.93 ms per token,    18.21 tokens per second)
llama_print_timings:        eval time =    8132.77 ms /   139 runs   (   58.51 ms per token,    17.09 tokens per second)
llama_print_timings:       total time =   10173.30 ms /   171 tokens
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [02:39<00:00,  6.38s/it]Llama.generate: prefix-match hit


  Based on the information provided in the article, I would estimate that the location being discussed is Boston, Massachusetts.

1. Y - The article is talking about Boston.
2. The specific location within Boston that I got is the North End neighborhood.


llama_print_timings:        load time =    8183.60 ms
llama_print_timings:      sample time =       4.55 ms /    54 runs   (    0.08 ms per token, 11857.71 tokens per second)
llama_print_timings: prompt eval time =    1912.09 ms /    34 tokens (   56.24 ms per token,    17.78 tokens per second)
llama_print_timings:        eval time =    3115.22 ms /    53 runs   (   58.78 ms per token,    17.01 tokens per second)
llama_print_timings:       total time =    5134.12 ms /    87 tokens
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [02:44<00:00,  6.10s/it]


#### Data Checking

In [21]:
df

,hl1,body,neighborhoods[0],llama_pred,token_length
0,Tonight‚Äôs Celebration of ‚ÄúEveryone‚Äôs Fav...,Tonight‚Äôs celebrity and family-filled progra...,Allston,"Sure, I'd be happy to help! Based on the inf...",3411
1,"Winter storm snarls travel, knocks out power f...",A storm making its way up the East Coast broug...,East Boston,Based on the information provided in the art...,2478
2,A Walk Down the Block: Nubian Square's economi...,"Yesterday, we took a walk down the block in Ro...",Roxbury,Here's my analysis:\n1. Y - The article is d...,2928
3,Mass. makes moves to figure out what happens a...,As the state nears its threshold level for hou...,South Boston Waterfront,1. Y - The article is talking about a region...,4245
4,‚ÄòWe need to be ready‚Äô: CDC gives Northeast...,Northeastern University will act as a national...,Fenway,"Sure, I'd be happy to help! Here's my respon...",1528
5,"Faced with a lack of funding, collegiate cheer...","For 10 Sundays each year, members of the Bridg...",Fenway,Based on the information provided in the art...,7539
6,How to know who deserves your vote in a news d...,Deception in politics is a tale as old as time...,Downtown,Based on the information provided in the art...,8300
7,Wu ‚Äòconfident‚Äô that new Mass. and Cass pla...,Boston Mayor Michelle Wu said late last week t...,Roxbury,Based on the information provided in the art...,4422
8,Healey anticipates ‚Äòhard calls‚Äô as emergen...,Gov. Maura Healey says she‚Äôs bracing to make...,Downtown,Based on the information provided in the art...,4609
9,"With need surging, behavioral urgent care cent...",East Boston Neighborhood Health Center held an...,East Boston,Based on the information provided in the art...,3296


### Data Cleaning on llama output

In [22]:
# Formats the output, Sorry for the disgusting lambda function
func_LlamaOutputFormatter = lambda x: ["".join([s.strip(), "."]) for s in (''.join(re.findall(r"[A-Za-z,:. ]", (x.split("2.")[1] if ("2." in x) else "N\A."))).split(".")) if (s != "")]

In [23]:
df['llama_post_process'] = df['llama_pred'].progress_apply(func_LlamaOutputFormatter)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 8869.53it/s]


In [24]:
df

,hl1,body,neighborhoods[0],llama_pred,token_length,llama_post_process
0,Tonight‚Äôs Celebration of ‚ÄúEveryone‚Äôs Fav...,Tonight‚Äôs celebrity and family-filled progra...,Allston,"Sure, I'd be happy to help! Based on the inf...",3411,[The specific location within the city I got i...
1,"Winter storm snarls travel, knocks out power f...",A storm making its way up the East Coast broug...,East Boston,Based on the information provided in the art...,2478,"[Cambridge, Massachusetts Based on the langua..."
2,A Walk Down the Block: Nubian Square's economi...,"Yesterday, we took a walk down the block in Ro...",Roxbury,Here's my analysis:\n1. Y - The article is d...,2928,[The specific location within the city I got i...
3,Mass. makes moves to figure out what happens a...,As the state nears its threshold level for hou...,South Boston Waterfront,1. Y - The article is talking about a region...,4245,[Based on the headline and the information pro...
4,‚ÄòWe need to be ready‚Äô: CDC gives Northeast...,Northeastern University will act as a national...,Fenway,"Sure, I'd be happy to help! Here's my respon...",1528,[The specific location within Boston that I in...
5,"Faced with a lack of funding, collegiate cheer...","For 10 Sundays each year, members of the Bridg...",Fenway,Based on the information provided in the art...,7539,[The specific location within Boston that I pi...
6,How to know who deserves your vote in a news d...,Deception in politics is a tale as old as time...,Downtown,Based on the information provided in the art...,8300,[South End Based on the mention of the South ...
7,Wu ‚Äòconfident‚Äô that new Mass. and Cass pla...,Boston Mayor Michelle Wu said late last week t...,Roxbury,Based on the information provided in the art...,4422,[The specific location within Boston is: Downt...
8,Healey anticipates ‚Äòhard calls‚Äô as emergen...,Gov. Maura Healey says she‚Äôs bracing to make...,Downtown,Based on the information provided in the art...,4609,[The specific location within Boston that I in...
9,"With need surging, behavioral urgent care cent...",East Boston Neighborhood Health Center held an...,East Boston,Based on the information provided in the art...,3296,[The specific location within East Boston that...


## LUKE Entity Recognition

In [25]:
import torch
import spacy
from transformers import LukeTokenizer, LukeModel, LukeForEntitySpanClassification

/Users/zgou/miniconda3/envs/SHIPLY_LLM/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
LUKE_model = LukeForEntitySpanClassification.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")
LUKE_tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")

Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-conll-2003 were not used when initializing LukeForEntitySpanClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
#sample_text = ["Based on the information provided in the article, I believe the location being described is the Omni Parker House hotel in Boston, Massachusetts, USA.", "This is very cool."]

In [28]:
def LUKE_EntityRecongition(text):
    """ 
    Given a sentence, we output an array of location that the LUKE recongizes in terms of LOC and ORG based on the IOB2 Sequence
    """
    # Convert to Spacy format
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    # Define the entity span based on the tokens, needed for LUKE
    entity_spans = []
    original_word_spans = []
    for token_start in doc:
        for token_end in doc[token_start.i:]:
            entity_spans.append((token_start.idx, token_end.idx + len(token_end)))
            original_word_spans.append((token_start.i, token_end.i + 1))
    
    inputs = LUKE_tokenizer(text, entity_spans=entity_spans, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = LUKE_model(**inputs)
    
    logits = outputs.logits

    max_logits, max_indices = logits[0].max(dim=1)
    
    predictions = []
    for logit, index, span in zip(max_logits, max_indices, original_word_spans):
        if index != 0:  # the span is not NIL
            predictions.append((logit, span, LUKE_model.config.id2label[int(index)]))
    
    # Construct an IOB2 label sequence
    predicted_sequence = ["O"] * len(doc)
    for _, span, label in sorted(predictions, key=lambda o: o[0], reverse=True):
        if all([o == "O" for o in predicted_sequence[span[0] : span[1]]]):
            predicted_sequence[span[0]] = "B-" + label
            if span[1] - span[0] > 1:
                predicted_sequence[span[0] + 1 : span[1]] = ["I-" + label] * (span[1] - span[0] - 1)

    # Linear scan for locations
    word_flag = False
    locations_and_orgs = []
    stringBuilder = ""
    
    for token_idx in range(len(predicted_sequence)):
        curr_seq = predicted_sequence[token_idx]
        curr_token = doc[token_idx]
        if ((curr_seq == 'B-ORG' or curr_seq == 'B-LOC') and word_flag == False):
            word_flag = True
            stringBuilder = stringBuilder + f" {curr_token}"
        elif ((curr_seq == 'B-ORG' or curr_seq == 'B-LOC') and word_flag == True): # Signifies new sequence, but attached to the one before
            locations_and_orgs.append(stringBuilder.strip())
            stringBuilder = "" # Reset and add
            stringBuilder = stringBuilder + f" {curr_token}"
        elif ((curr_seq != 'O') and word_flag == True):
            stringBuilder = stringBuilder + f" {curr_token}"
        elif ((curr_seq == 'O') and word_flag == True):
            word_flag = False
            locations_and_orgs.append(stringBuilder.strip())
            stringBuilder = "" # Reset and add
    
        # Check if I-XXX is the last token
        if ((token_idx == len(predicted_sequence) - 1) and stringBuilder != ""):
            locations_and_orgs.append(stringBuilder.trim())
            stringBuilder = ""
    return locations_and_orgs

def predictFlatten(arr_inputs, func):
    """
    Given an array of inputs, we use a function to get another array and then flatten all the results into one arr
    """
    res = []
    for x in arr_inputs:
        res.append(func(x))
    return [j for sub in res for j in sub] # We flatten here

In [29]:
func_LUKEPredictions = lambda x: predictFlatten(x, LUKE_EntityRecongition)

In [30]:
df['LUKE_pred'] = df['llama_post_process'].progress_apply(func_LUKEPredictions)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:36<00:00,  1.34s/it]


In [31]:
df

,hl1,body,neighborhoods[0],llama_pred,token_length,llama_post_process,LUKE_pred
0,Tonight‚Äôs Celebration of ‚ÄúEveryone‚Äôs Fav...,Tonight‚Äôs celebrity and family-filled progra...,Allston,"Sure, I'd be happy to help! Based on the inf...",3411,[The specific location within the city I got i...,"[North End, North End, Boston]"
1,"Winter storm snarls travel, knocks out power f...",A storm making its way up the East Coast broug...,East Boston,Based on the information provided in the art...,2478,"[Cambridge, Massachusetts Based on the langua...","[Cambridge, Massachusetts, Cambridge, Massachu..."
2,A Walk Down the Block: Nubian Square's economi...,"Yesterday, we took a walk down the block in Ro...",Roxbury,Here's my analysis:\n1. Y - The article is d...,2928,[The specific location within the city I got i...,"[Roxbury, Boston, Nubian Square, Roxbury, Boston]"
3,Mass. makes moves to figure out what happens a...,As the state nears its threshold level for hou...,South Boston Waterfront,1. Y - The article is talking about a region...,4245,[Based on the headline and the information pro...,"[Massachusetts, Boston]"
4,‚ÄòWe need to be ready‚Äô: CDC gives Northeast...,Northeastern University will act as a national...,Fenway,"Sure, I'd be happy to help! Here's my respon...",1528,[The specific location within Boston that I in...,"[Boston, Boston University, Boston Medical Cen..."
5,"Faced with a lack of funding, collegiate cheer...","For 10 Sundays each year, members of the Bridg...",Fenway,Based on the information provided in the art...,7539,[The specific location within Boston that I pi...,"[Boston, Fenway]"
6,How to know who deserves your vote in a news d...,Deception in politics is a tale as old as time...,Downtown,Based on the information provided in the art...,8300,[South End Based on the mention of the South ...,"[South End, South End, Boston]"
7,Wu ‚Äòconfident‚Äô that new Mass. and Cass pla...,Boston Mayor Michelle Wu said late last week t...,Roxbury,Based on the information provided in the art...,4422,[The specific location within Boston is: Downt...,"[Boston, Downtown Boston, Massachusetts Avenue..."
8,Healey anticipates ‚Äòhard calls‚Äô as emergen...,Gov. Maura Healey says she‚Äôs bracing to make...,Downtown,Based on the information provided in the art...,4609,[The specific location within Boston that I in...,[]
9,"With need surging, behavioral urgent care cent...",East Boston Neighborhood Health Center held an...,East Boston,Based on the information provided in the art...,3296,[The specific location within East Boston that...,[East Boston]


### Optionally Concat to Singer Eval

### Export

In [32]:
df.to_csv("./outputs/test_3.csv")